In [1]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [3]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [8]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/MyDrive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "StI0nAmwRZyaqxHnMGZk6oIeE"
    consumer_secret = "QVrZ8kmOZA1E9lWqeExBxe75JbOSywBXoVlLjvW0rAvsszuFj4"
    access_token = "1514294618019426305-FhmJEyF5xlXxxmmjtySzgE2Xx6Nu4y"
    access_token_secret = "swn4hw2x0qhoKL9inOL4XkPv2Vh3aAyRRuHQLYnWFiGJg"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-74.1687, 40.5722, -73.8062, 40.9467,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS)

{'id': 1524535463813554176, 'username': 'Jay_Hoffa_Flee', 'created_at': 'Wed May 11 23:42:33 +0000 2022', 'lng': -73.9487755, 'lat': 40.655138, 'text': 'Fresh clean watered Blackberries and Strawberries chilling'}
{'id': 1524535465864581121, 'username': 'Aissathebeergod', 'created_at': 'Wed May 11 23:42:34 +0000 2022', 'lng': -73.8311875, 'lat': 40.6708795, 'text': 'Sometimes I want you but most times I want nothing to do with you'}
{'id': 1524535480280395776, 'username': 'famousboyfriend', 'created_at': 'Wed May 11 23:42:37 +0000 2022', 'lng': -73.9487755, 'lat': 40.655138, 'text': 'mister fraud'}
{'id': 1524535482020999169, 'username': 'je_mclaughlin', 'created_at': 'Wed May 11 23:42:38 +0000 2022', 'lng': -74.7243235, 'lat': 40.073040500000005, 'text': '@mschlapp Poor @mercedesschlapp'}
{'id': 1524535487217778688, 'username': 'GerardAraud', 'created_at': 'Wed May 11 23:42:39 +0000 2022', 'lng': -73.9685415, 'lat': 40.780709, 'text': '@AntonJo39260714 Oh my God? Really ?'}
{'id': 152

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
